In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Loding Dataset #

In [2]:
def load_data(data_dir):
    images = []
    labels = []
    label_map = {'Organic': 0, 'Recyclable': 1}

    for label in os.listdir(data_dir):
        if label in label_map:
            for img_file in os.listdir(os.path.join(data_dir, label)):
                img_path = os.path.join(data_dir, label, img_file)
                image = cv2.imread(img_path)
                image = cv2.resize(image, (224, 224))  
                images.append(image)
                labels.append(label_map[label])

    return np.array(images), np.array(labels)

# Load the dataset
X, y = load_data('dataset/train')

# Normalize pixel values
X = X / 255.0  

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Traning Neural Network #

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [5]:

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  # 2 classes: organic and recyclable

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()

c:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [6]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.6970 - val_accuracy: 0.4286 - val_loss: 1.1102
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - accuracy: 0.6250 - loss: 1.6562 - val_accuracy: 0.5714 - val_loss: 2.2364
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step - accuracy: 0.5000 - loss: 3.3336 - val_accuracy: 0.4286 - val_loss: 1.5510
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step - accuracy: 0.7083 - loss: 1.3198 - val_accuracy: 0.4286 - val_loss: 0.9841
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step - accuracy: 0.5833 - loss: 0.6995 - val_accuracy: 0.8571 - val_loss: 0.5465
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step - accuracy: 0.9583 - loss: 0.3690 - val_accuracy: 0.8571 - val_loss: 0.4967
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step - accuracy: 0.8750 - loss: 0.3534 - val_accuracy: 0.7143 - val_loss: 0.5183
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.9583 - loss: 0.2804 - val_accuracy: 0.4286 - val_loss: 0.

# Save these Model #

In [7]:
model.save('waste_classification_model.h5')

# Display Model result in Commnad Line using image #

In [8]:
def classify_waste(Dataset, model, threshold=0.5):
    image = cv2.imread(Dataset)
    image = cv2.resize(image, (224, 224))  

    # Normalize
    image = np.expand_dims(image, axis=0) / 255.0  

    prediction = model.predict(image)
    class_index = np.argmax(prediction)

    # Check if the prediction confidence is above the threshold
    if np.max(prediction) < threshold:
        return "NO RECORD FOUND"
    else:
        return "Organic Waste" if class_index == 0 else "Recyclable Waste"

result = classify_waste("Dataset\Test\Organic\WhatsApp Image 2025-03-16 at 7.52.28 PM.jpeg", model)
print(result)

<>:17: SyntaxWarning: invalid escape sequence '\T'
<>:17: SyntaxWarning: invalid escape sequence '\T'
C:\Users\ayush\AppData\Local\Temp\ipykernel_14888\3877082635.py:17: SyntaxWarning: invalid escape sequence '\T'
  result = classify_waste("Dataset\Test\Organic\WhatsApp Image 2025-03-16 at 7.52.28 PM.jpeg", model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Organic Waste


# Display Model result in Commnad Line using Camera #

In [9]:
from tensorflow.keras.models import load_model


model = load_model('waste_classification_model.h5')

# Function to classify waste
def classify_waste(image, model, threshold=0.5):
    image_resized = cv2.resize(image, (224, 224))  
    image_normalized = np.expand_dims(image_resized, axis=0) / 255.0 

    prediction = model.predict(image_normalized)
    class_index = np.argmax(prediction)

    if np.max(prediction) < threshold:
        return "Unknown"
    else:
        return "Organic" if class_index == 0 else "Recyclable"

# Initialize the camera
# Use 0 for the default camera
cap = cv2.VideoCapture(0)  

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Display the frame
    cv2.imshow("Camera Feed", frame)

    # Wait for the user to press 'c' to classify the current frame
    key = cv2.waitKey(1)
    if key & 0xFF == ord('c'):
        result = classify_waste(frame, model)
        print(f"Classification Result: {result}")

    # Exit the loop when 'q' is pressed
    if key & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Classification Result: Recyclable
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Classification Result: Organic
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Classification Result: Recyclable


# Display Model result in Frame using camera #

In [10]:
model = load_model('waste_classification_model.h5')

def classify_waste(image, model, threshold=0.5):
    image_resized = cv2.resize(image, (224, 224))  
    image_normalized = np.expand_dims(image_resized, axis=0) / 255.0  

    prediction = model.predict(image_normalized)
    class_index = np.argmax(prediction)

    if np.max(prediction) < threshold:
        return "Unknown"
    else:
        return "Organic" if class_index == 0 else "Recyclable"


cap = cv2.VideoCapture(0)  

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    cv2.imshow("Camera Feed", frame)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('c'):
        result = classify_waste(frame, model)
        print(f"Classification Result: {result}")

        result_window = np.zeros((200, 400, 3), dtype=np.uint8) 
        cv2.putText(result_window, f"Result: {result}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow("Classification Result", result_window)

        
        cv2.waitKey(3000) 
        cv2.destroyWindow("Classification Result")  

    if key & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Classification Result: Recyclable
